<a href="https://colab.research.google.com/github/Samin-Sadaf7/Book-of-LLMs/blob/main/HandsOnLLM_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
#Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
#Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

Device set to use cuda


In [4]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened"
output = generator(prompt)
print(output[0]['generated_text'])

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


, express your sincere regret, and offer to make amends by helping her replant the flowers.

Dear Sarah,

I hope this email finds you well. I am writing to express my deepest apologies for the


In [5]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

In [7]:
prompt ="The Capital of France is"

#Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
print(input_ids)
input_ids = input_ids.to("cuda")
print(input_ids)

tensor([[  450, 25343,   310,  3444,   338]])
tensor([[  450, 25343,   310,  3444,   338]], device='cuda:0')


In [10]:
#Get the output of the model before lm head
model_output = model.model(input_ids)
#Get the output of the model head
lm_head_output = model.lm_head(model_output[0])

In [13]:
lm_head_output.shape

torch.Size([1, 5, 32064])

In [18]:
lm_head_output[0,-1]

tensor([28.6250, 31.0000, 28.8750,  ..., 22.8750, 22.8750, 22.8750],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SelectBackward0>)

In [15]:
token_id = lm_head_output[0,-1].argmax(-1)
token_id

tensor(3681, device='cuda:0')

In [16]:
tokenizer.decode(token_id)

'Paris'

In [19]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [20]:
lm_head_output.shape

torch.Size([1, 5, 32064])

In [21]:
prompt="Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened"
#Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [23]:
%%timeit -n 1
#Generate the text using cache for middle transformer blocks attention machines
#mainly check kv cache (keys and values cache)
generate_output = model.generate(
    input_ids = input_ids,
    max_new_tokens=100,
    use_cache=True
)

6.61 s ± 1.46 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit -n 1
#Generate the text using cache for middle transformer blocks attention machines
#mainly check kv cache (keys and values cache)
generate_output = model.generate(
    input_ids = input_ids,
    max_new_tokens=100,
    use_cache=False
)

32.6 s ± 203 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
